## Preprocess data

In [ ]:
import pandas as pd
import numpy as np

In [18]:

# Gets modifier value for each stat
def getModifier(stat, baseModifier=-5, startingStat=1, highestStat=30):
    modifier = baseModifier
    # if stat is equal to the base value return base modifier
    if stat == startingStat: return modifier

    # increment modifier by 1
    modifier += 1
    # Loop through all possible stat values and return the corresponding modifier value
    for i in range(2, highestStat):
        # if stat is between these values return the related modifier
        if stat >= i and stat <= i + 1: return modifier
        if not i % 2 == 1: modifier += 1
    return modifier

for val in range(1, 31):
    print(f"{val}: {getModifier(val)}")

1: -5
2: -4
3: -4
4: -3
5: -3
6: -2
7: -2
8: -1
9: -1
10: 0
11: 0
12: 1
13: 1
14: 2
15: 2
16: 3
17: 3
18: 4
19: 4
20: 5
21: 5
22: 6
23: 6
24: 7
25: 7
26: 8
27: 8
28: 9
29: 9
30: 10
